In [21]:
import pandas as pd
import numpy as np
import requests
from underdog_scraper import UnderdogScraper
import re
from datetime import datetime





In [3]:
def get_pp_projections():
    # Define the URL and headers as in the curl request
    url = 'https://api.prizepicks.com/projections?league_id=159&per_page=250&single_stat=true&in_game=true&state_code=CA&game_mode=pickem'

    headers = {
        'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Referer': 'https://app.prizepicks.com/',
        'X-Device-ID': '1a9d6304-65f3-4304-8523-ccf458d3c0c4',
        'sec-ch-ua-platform': '"macOS"'
    }

    # Send the GET request
    response = requests.get(url, headers=headers)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

    opponents = []
    player_ids = []
    stat_type = []
    line = []
    for item in data['data']:
        player_info = item['relationships']
        player_id = player_info['new_player']['data']['id']
        player_ids.append(player_id)
        item = item['attributes']
        opponents.append(item['description'])
        stat_type.append(item['stat_type'])
        line.append(item['line_score'])

    players = []
    for id in player_ids:
        for item in data['included']:
            if item['type'] == 'new_player':
                if item['id'] == id:
                    players.append(item['attributes']['name'])
    return pd.DataFrame({
        'players': players,
        'opponents': opponents,
        'stat_type': stat_type,
        'line': line
        })

In [4]:
pp_projections = get_pp_projections()
pp_projections['Sportsbook'] = 'Prizepicks'

In [5]:
scraper = UnderdogScraper()
underdog_projections = scraper.scrape()
underdog_projections = underdog_projections[underdog_projections['full_name'].str.contains('Val', na = False) & ((underdog_projections['sport_id'] == 'ESPORTS') | (underdog_projections['sport_id'] == 'VAL'))]
underdog_projections =  underdog_projections[['sport_id', 'stat_name', 'stat_value',
                             'full_name', 'last_name', 'american_price', 'decimal_price', 'payout_multiplier']]
underdog_projections['Sportsbook'] = 'Underdog'




Data saved to underdog_props.csv


In [9]:
pp_projections

,players,opponents,stat_type,line,Sportsbook
0,artzin,Evil Geniuses MAP 3,MAP 3 Kills,14.0,Prizepicks
1,xenom,Evil Geniuses MAP 3,MAP 3 Kills,14.5,Prizepicks
2,Cortezia,Evil Geniuses MAP 3,MAP 3 Kills,15.0,Prizepicks
3,aspas,Evil Geniuses MAP 3,MAP 3 Kills,18.0,Prizepicks
4,Verno,Evil Geniuses MAP 3,MAP 3 Kills,14.5,Prizepicks
5,Derrek,MIBR MAP 3,MAP 3 Kills,14.0,Prizepicks
6,NaturE,MIBR MAP 3,MAP 3 Kills,12.5,Prizepicks
7,supamen,MIBR MAP 3,MAP 3 Kills,14.5,Prizepicks
8,yay,MIBR MAP 3,MAP 3 Kills,16.5,Prizepicks
9,MaKo,Paper Rex MAPS 1-3,MAPS 1-3 Kills,48.5,Prizepicks


In [7]:
underdog_projections

,sport_id,stat_name,stat_value,full_name,last_name,american_price,decimal_price,payout_multiplier,Sportsbook
7708,ESPORTS,kills_on_maps_1_2_3,50.5,Val: kamo,kamo,-112,1.9,1.0,Underdog
7709,ESPORTS,kills_on_maps_1_2_3,50.5,Val: kamo,kamo,-112,1.9,1.0,Underdog
7710,ESPORTS,kills_on_maps_1_2_3,51.5,Val: marteen,marteen,-112,1.9,1.0,Underdog
7711,ESPORTS,kills_on_maps_1_2_3,51.5,Val: marteen,marteen,-112,1.9,1.0,Underdog
7762,ESPORTS,kills_on_maps_1_2_3,49.5,Val: Keiko,Keiko,-112,1.9,1.0,Underdog
7763,ESPORTS,kills_on_maps_1_2_3,49.5,Val: Keiko,Keiko,-112,1.9,1.0,Underdog
7776,ESPORTS,kills_on_maps_1_2_3,41.5,Val: paTiTek,paTiTek,-112,1.9,1.0,Underdog
7777,ESPORTS,kills_on_maps_1_2_3,41.5,Val: paTiTek,paTiTek,-112,1.9,1.0,Underdog
7834,ESPORTS,kills_on_maps_1_2_3,54.5,Val: HYUNMIN,HYUNMIN,-112,1.9,1.0,Underdog
7835,ESPORTS,kills_on_maps_1_2_3,54.5,Val: HYUNMIN,HYUNMIN,-112,1.9,1.0,Underdog


In [12]:
# --- Step 1: Clean pp_projections (PrizePicks) ---
pp_cleaned = pp_projections.copy()

# Standardize column names
pp_cleaned = pp_cleaned.rename(columns={
    "players": "player_name",
    "opponents": "opponent",
    "line": "line",
    "stat_type": "stat_type",
    "Sportsbook": "sportsbook"
})

# Normalize player names
pp_cleaned["player_name"] = pp_cleaned["player_name"].str.strip().str.lower()

# Normalize stat_type
def normalize_stat_type(stat):
    stat = stat.lower().strip()
    stat = stat.replace("map ", "map_").replace("maps ", "maps_")
    stat = stat.replace(" ", "_")

    # Optional regex cleanup for patterns like kills_on_maps_1_2_3
    stat = re.sub(r'kills_on_maps_(\d+)_?(\d+)?_?(\d+)?', 
                  lambda m: f"maps_{m.group(1)}_{m.group(2)}_{m.group(3)}_kills" if m.group(3) 
                            else f"maps_{m.group(1)}_{m.group(2)}_kills", 
                  stat)
    return stat

pp_cleaned["stat_type"] = pp_cleaned["stat_type"].apply(normalize_stat_type)


# --- Step 2: Clean underdog_projections ---
underdog_cleaned = underdog_projections.copy()

# Standardize column names and keep only relevant ones
underdog_cleaned = underdog_cleaned.rename(columns={
    "last_name": "player_name",
    "stat_name": "stat_type",
    "stat_value": "line",
    "Sportsbook": "sportsbook"
})

underdog_cleaned["player_name"] = underdog_cleaned["player_name"].str.strip().str.lower()
underdog_cleaned["stat_type"] = underdog_cleaned["stat_type"].apply(normalize_stat_type)

# Add opponent column as NaN to match pp_cleaned structure
underdog_cleaned["opponent"] = None

# Select relevant columns to match pp_cleaned
underdog_cleaned = underdog_cleaned[["player_name", "opponent", "stat_type", "line", "sportsbook"]]


# --- Step 3: Combine the cleaned dataframes ---
combined_projections = pd.concat([pp_cleaned, underdog_cleaned], ignore_index=True)

# Optional: sort or reset index
combined_projections = combined_projections.sort_values(by=["stat_type", "player_name"]).reset_index(drop=True)





  player_name             opponent    stat_type  line  sportsbook
0      artzin  Evil Geniuses MAP 3  map_3_kills  14.0  Prizepicks
1       aspas  Evil Geniuses MAP 3  map_3_kills  18.0  Prizepicks
2    cortezia  Evil Geniuses MAP 3  map_3_kills  15.0  Prizepicks
3      derrek           MIBR MAP 3  map_3_kills  14.0  Prizepicks
4      nature           MIBR MAP 3  map_3_kills  12.5  Prizepicks


In [22]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"valorant_projections_{timestamp}.csv"

combined_projections.to_csv(filename, index = False)